In [ ]:
# Как подружить Python и базы данных SQL. Подробное руководство
# https://proglib.io/p/kak-podruzhit-python-i-bazy-dannyh-sql-podrobnoe-rukovodstvo-2020-02-27

In [ ]:
# 2. Подключение к базам данных

In [ ]:
# mySQL

# В отличие от SQLite, в Python по умолчанию нет модуля, который можно использовать для подключения к базе данных MySQL.
# Для этого вам нужно установить драйвер Python для MySQL. Одним из таких драйверов является mysql-connector-python.
# Вы можете скачать этот модуль Python SQL с помощью pip:
        
# pip install mysql-connector-python

In [ ]:
# Обратите внимание, что MySQL – это серверная система управления базами данных.
# Один сервер MySQL может хранить несколько баз данных.
# В отличие от SQLite, где соединение равносильно порождению БД, формирование базы данных MySQL состоит из двух этапов:

# Установка соединения с сервером MySQL.
# Выполнение запроса для создания БД.
# Определим функцию, которая будет подключаться к серверу MySQL и возвращать объект подключения:

In [ ]:
import mysql.connector
from mysql.connector import Error

def create_connection(host_name, user_name, user_password):
    connection = None
    try:
        connection = mysql.connector.connect(
            host=host_name,
            user=user_name,
            passwd=user_password
        )
        print("Connection to MySQL DB successful")
    except Error as e:
        print(f"The error '{e}' occurred")

    return connection

connection = create_connection("localhost", "root", "")

In [ ]:
# В приведенном выше коде мы определили новую функцию create_connection(), которая принимает три параметра:

# host_name
# user_name
# user_password

# Модуль mysql.connector определяет метод connect(), используемый в седьмой строке для подключения к серверу MySQL.
# Как только соединение установлено, объект connection возвращается вызывающей функции.
# В последней строке функция create_connection() вызывается с именем хоста, именем пользователя и паролем.

# Пока мы только установили соединение. Самой базы ещё нет.
# Для этого мы определим другую функцию – create_database(), которая принимает два параметра:

# Объект connection;
# query – строковый запрос о создании базу данных.
# Вот как выглядит эта функция:

In [ ]:
def create_database(connection, query):
    cursor = connection.cursor()
    try:
        cursor.execute(query)
        print("Database created successfully")
    except Error as e:
        print(f"The error '{e}' occurred")

In [ ]:
# Для выполнения запросов используется объект cursor.
# Создадим базу данных sm_appдля нашего приложения на сервере MySQL:
        
create_database_query = "CREATE DATABASE sm_app"
create_database(connection, create_database_query)

In [ ]:
# Теперь у нас есть база данных на сервере. Однако объект connection,
# возвращаемый функцией create_connection() подключен к серверу MySQL.
# А нам необходимо подключиться к базе данных sm_app.
# Для этого нужно изменить create_connection() следующим образом:

In [ ]:
def create_connection(host_name, user_name, user_password, db_name):
    connection = None
    try:
        connection = mysql.connector.connect(
            host=host_name,
            user=user_name,
            passwd=user_password,
            database=db_name
        )
        print("Connection to MySQL DB successful")
    except Error as e:
        print(f"The error '{e}' occurred")

    return connection

In [ ]:
# Функция create_connection() теперь принимает дополнительный параметр с именем db_name.
# Этот параметр указывает имя БД, к которой мы хотим подключиться. Имя теперь можно передать при вызове функции:
        
connection = create_connection("localhost", "root", "", "sm_app")

# Скрипт успешно вызывает create_connection() и подключается к базе данных sm_app.

In [ ]:
# 3. Создание таблиц
# В предыдущем разделе мы увидели, как подключаться к серверам баз данных SQLite, MySQL и PostgreSQL, 
# используя разные библиотеки Python. Мы создали базу данных sm_app на всех трех серверах БД.
# В данном разделе мы рассмотрим, как формировать таблицы внутри этих трех баз данных.

# Как обсуждалось ранее, нам нужно получить и связать четыре таблицы:

# users
# posts
# comments
# likes

In [ ]:
# MySQL
# Так же, как с SQLite, чтобы создать таблицу в MySQL, нужно передать запрос в cursor.execute().
# Создадим новый вариант функции execute_query():

In [ ]:
def execute_query(connection, query):
     cursor = connection.cursor()
     try:
         cursor.execute(query)
         connection.commit()
         print("Query executed successfully")
     except Error as e:
         print(f"The error '{e}' occurred")

In [ ]:
# Описываем таблицу users:

        
create_users_table = """
CREATE TABLE IF NOT EXISTS users (
  id INT AUTO_INCREMENT, 
  name TEXT NOT NULL, 
  age INT, 
  gender TEXT, 
  nationality TEXT, 
  PRIMARY KEY (id)
) ENGINE = InnoDB
"""

execute_query(connection, create_users_table)

In [ ]:
# Запрос для реализации отношения внешнего ключа в MySQL немного отличается от SQLite.
# Более того, MySQL использует ключевое слово AUTO_INCREMENT для указания столбцов,
# значения которых автоматически увеличиваются при вставке новых записей.

# Следующий скрипт составит таблицу posts, содержащую внешний ключ user_id, который ссылается на id столбца таблицы users:

In [ ]:
create_posts_table = """
CREATE TABLE IF NOT EXISTS posts (
  id INT AUTO_INCREMENT, 
  title TEXT NOT NULL, 
  description TEXT NOT NULL, 
  user_id INTEGER NOT NULL, 
  FOREIGN KEY fk_user_id (user_id) REFERENCES users(id), 
  PRIMARY KEY (id)
) ENGINE = InnoDB
"""

execute_query(connection, create_posts_table)

In [ ]:
# Аналогично для создания таблиц comments и likes, передаём соответствующие CREATE-запросы функции execute_query().

In [ ]:
# 4. Вставка записей
# В предыдущем разделе мы разобрали, как развертывать таблицы в базах данных SQLite, MySQL и PostgreSQL
# с использованием различных модулей Python. В этом разделе узнаем, как вставлять записи.

In [ ]:
# MySQL
# Есть два способа вставить записи в базы данных MySQL из приложения Python. Первый подход похож на SQLite.
# Можно сохранить запрос INSERT INTO в строке, а затем использовать для вставки записей cursor.execute().

# Ранее мы определили функцию-оболочку execute_query(), которую использовали для вставки записей.
# Мы можем использовать ту же функцию:

In [ ]:
create_users = """
INSERT INTO
  `users` (`name`, `age`, `gender`, `nationality`)
VALUES
  ('James', 25, 'male', 'USA'),
  ('Leila', 32, 'female', 'France'),
  ('Brigitte', 35, 'female', 'England'),
  ('Mike', 40, 'male', 'Denmark'),
  ('Elizabeth', 21, 'female', 'Canada');
"""

execute_query(connection, create_users)  

In [ ]:
# Второй подход использует метод cursor.executemany(), который принимает два параметра:

# Строка query, содержащая заполнители для вставляемых записей.
# Список записей, которые мы хотим вставить.
# Посмотрите на следующий пример, который вставляет две записи в таблицу likes:

In [ ]:
sql = "INSERT INTO likes ( user_id, post_id ) VALUES ( %s, %s )"
val = [(4, 5), (3, 4)]

cursor = connection.cursor()
cursor.executemany(sql, val)
connection.commit()

    
# Какой подход выбрать – зависит от вас. Если вы не очень хорошо знакомы с SQL, проще использовать метод курсора executemany().

In [ ]:
# 5. Извлечение данных из записей

In [ ]:
# SQLite
# Чтобы выбрать записи в SQLite, можно снова использовать cursor.execute().
# Однако после этого потребуется вызвать метод курсора fetchall().
# Этот метод возвращает список кортежей, где каждый кортеж сопоставлен с соответствующей строкой в ​​извлеченных записях.
# Чтобы упростить процесс, напишем функцию execute_read_query():

        
def execute_read_query(connection, query):
    cursor = connection.cursor()
    result = None
    try:
        cursor.execute(query)
        result = cursor.fetchall()
        return result
    except Error as e:
        print(f"The error '{e}' occurred")
        
# Эта функция принимает объект connection и SELECT-запрос, а возвращает выбранную запись.

In [ ]:
# SELECT
# Давайте выберем все записи из таблицы users:

        
select_users = "SELECT * from users"
users = execute_read_query(connection, select_users)

for user in users:
    print(user)

In [ ]:
# В приведенном выше скрипте запрос SELECT забирает всех пользователей из таблицы users.
# Результат передается в написанную нами функцию execute_read_query(), возвращающую все записи из таблицы users.

# Примечание. Не рекомендуется использовать SELECT * для больших таблиц,
# так как это может привести к большому числу операций ввода-вывода, которые увеличивают сетевой трафик.

# Результат вышеприведенного запроса выглядит следующим образом:

        
# (1, 'James', 25, 'male', 'USA')
# (2, 'Leila', 32, 'female', 'France')
# (3, 'Brigitte', 35, 'female', 'England')
# (4, 'Mike', 40, 'male', 'Denmark')
# (5, 'Elizabeth', 21, 'female', 'Canada')

In [ ]:
# Таким же образом вы можете извлечь все записи из таблицы posts:

        
select_posts = "SELECT * FROM posts"
posts = execute_read_query(connection, select_posts)

for post in posts:
    print(post)

    
# Вывод выглядит так:

        
# (1, 'Happy', 'I am feeling very happy today', 1)
# (2, 'Hot Weather', 'The weather is very hot today', 2)
# (3, 'Help', 'I need some help with my work', 2)
# (4, 'Great News', 'I am getting married', 1)
# (5, 'Interesting Game', 'It was a fantastic game of tennis', 5)
# (6, 'Party', 'Anyone up for a late-night party today?', 3)

In [ ]:
# JOIN
# Вы также можете выполнять более сложные запросы, включающие операции типа JOIN для извлечения данных из двух связанных таблиц.
# Например, следующий скрипт возвращает идентификаторы и имена пользователей, а также описание сообщений,
# опубликованных этими пользователями:

In [ ]:
select_users_posts = """
SELECT
  users.id,
  users.name,
  posts.description
FROM
  posts
  INNER JOIN users ON users.id = posts.user_id
"""

users_posts = execute_read_query(connection, select_users_posts)

for users_post in users_posts:
    print(users_post)

In [ ]:
# Вывод данных:

        
# (1, 'James', 'I am feeling very happy today')
# (2, 'Leila', 'The weather is very hot today')
# (2, 'Leila', 'I need some help with my work')
# (1, 'James', 'I am getting married')
# (5, 'Elizabeth', 'It was a fantastic game of tennis')
# (3, 'Brigitte', 'Anyone up for a late night party today?')

In [ ]:
# Следующий скрипт возвращает все сообщения вместе с комментариями к сообщениям и именами пользователей,
# которые разместили комментарии:

        
select_posts_comments_users = """
SELECT
  posts.description as post,
  text as comment,
  name
FROM
  posts
  INNER JOIN comments ON posts.id = comments.post_id
  INNER JOIN users ON users.id = comments.user_id
"""

posts_comments_users = execute_read_query(
    connection, select_posts_comments_users
)

for posts_comments_user in posts_comments_users:
    print(posts_comments_user)

In [ ]:
# Вывод выглядит так:

        
# ('Anyone up for a late night party today?', 'Count me in', 'James')
# ('I need some help with my work', 'What sort of help?', 'Elizabeth')
# ('I am getting married', 'Congrats buddy', 'Leila')
# ('It was a fantastic game of tennis', 'I was rooting for Nadal though', 'Mike')
# ('I need some help with my work', 'Help with your thesis?', 'Leila')
# ('I am getting married', 'Many congratulations', 'Elizabeth')

In [ ]:
# Из вывода понятно, что имена столбцов не были возвращены методом fetchall().
# Чтобы вернуть имена столбцов, нужно забрать атрибут description объекта cursor.
# Например, следующий список возвращает все имена столбцов для вышеуказанного запроса:

        
cursor = connection.cursor()
cursor.execute(select_posts_comments_users)
cursor.fetchall()

column_names = [description[0] for description in cursor.description]
print(column_names)

    
# Вывод выглядит так:

        
# ['post', 'comment', 'name']

In [ ]:
# WHERE
# Теперь мы выполним SELECT-запрос, который возвращает текст поста и общее количество лайков, им полученных:

In [ ]:
select_post_likes = """
SELECT
  description as Post,
  COUNT(likes.id) as Likes
FROM
  likes,
  posts
WHERE
  posts.id = likes.post_id
GROUP BY
  likes.post_id
"""

post_likes = execute_read_query(connection, select_post_likes)

for post_like in post_likes:
    print(post_like)

In [ ]:
# Вывод следующий:

        
# ('The weather is very hot today', 1)
# ('I need some help with my work', 1)
# ('I am getting married', 2)
# ('It was a fantastic game of tennis', 1)
# ('Anyone up for a late night party today?', 2)

    
# То есть используя запрос WHERE, вы можете возвращать более конкретные результаты.

In [ ]:
# 5. Извлечение данных из записей
# MySQL
# Процесс выбора записей в MySQL абсолютно идентичен процессу выбора записей в SQLite:

        
def execute_read_query(connection, query):
    cursor = connection.cursor()
    result = None
    try:
        cursor.execute(query)
        result = cursor.fetchall()
        return result
    except Error as e:
        print(f"The error '{e}' occurred")

In [ ]:
# Теперь выберем все записи из таблицы users:

        
select_users = "SELECT * FROM users"
users = execute_read_query(connection, select_users)

for user in users:
    print(user)

    
# Вывод будет похож на то, что мы видели с SQLite.

In [ ]:
# 6. Обновление записей таблицы

In [ ]:
# SQLite
# Обновление записей в SQLite выглядит довольно просто. Снова можно применить execute_query().
# В качестве примера обновим текст поста с id равным 2. Сначала создадим описание для SELECT:

        
select_post_description = "SELECT description FROM posts WHERE id = 2"

post_description = execute_read_query(connection, select_post_description)

for description in post_description:
    print(description)

In [ ]:
# Увидим следующий вывод:

        
# ('The weather is very hot today',)

In [ ]:
# Следующий скрипт обновит описание:

        
update_post_description = """
UPDATE
  posts
SET
  description = "The weather has become pleasant now"
WHERE
  id = 2
"""

execute_query(connection, update_post_description)

In [ ]:
# Теперь, если мы выполним SELECT-запрос еще раз, увидим следующий результат:

        
# ('The weather has become pleasant now',)

    
# То есть запись была обновлена.

In [ ]:
# 6. Обновление записей таблицы
# MySQL
# Процесс обновления записей в MySQL с помощью модуля mysql-connector-python является точной копией модуля sqlite3:

In [ ]:
update_post_description = """
UPDATE
  posts
SET
  description = "The weather has become pleasant now"
WHERE
  id = 2
"""

execute_query(connection,  update_post_description)

In [ ]:
# 7. Удаление записей таблицы
# SQLite
# В качестве примера удалим комментарий с id равным 5:

        
delete_comment = "DELETE FROM comments WHERE id = 5"
execute_query(connection, delete_comment)

In [ ]:
# Теперь, если мы извлечем все записи из таблицы comments, то увидим, что пятый комментарий был удален.
# Процесс удаления в MySQL и PostgreSQL идентичен SQLite:

In [ ]:
# Заключение
# В этом руководстве мы разобрались, как применять три распространенные библиотеки Python для работы с реляционными базами данных.
# Научившись работать с одним из модулей sqlite3, mysql-connector-python и psycopg2,
# вы легко сможете перенести свои знания на другие модули и оперировать любой из баз данных SQLite, MySQL и PostgreSQL.

# Однако это лишь вершина айсберга! Существуют также библиотеки для работы с SQL и объектно-реляционными отображениями,
# такие как SQLAlchemy и Django ORM, которые автоматизируют задачи взаимодействия Python с базами данных.